## Mindspore自动微分

自动微分能够计算可导函数在某点处的导数值，是反向传播算法的一般化。自动微分主要解决的问题是将一个复杂的数学运算分解为一系列简单的基本运算，该功能对用户屏蔽了大量的求导细节和过程，大大降低了框架的使用门槛。

MindSpore使用**ops.GradOperation**计算一阶导数，**ops.Gradoperation**的示例如下：
$$f(x)=\omega x +b$$

可以构造一个Net类，w取6，b取1

$$f(x)=6x+1$$

In [26]:
import mindspore as ms
import numpy as np
import mindspore.nn as nn

class Net(nn.Cell):
    def __init__(self):
        super(Net,self).__init__()
        self.w=ms.Parameter(ms.Tensor(np.array([6],np.float32)),name='w')
        self.b=ms.Parameter(ms.Tensor(np.array([1],np.float32)),name='b')
    
    def construct(self,x):
        f=self.w*x+self.b
        return f

gs=Net() # 创建公式实例
print(gs(2))  # 传入x的值，进行推理


[13.]


Mindspore里的construct类似于forward，可以用前向调用。可以看到当我们传入x等于2时候，值为14，公式已经构建完成了，接下来就是进行自动微分的操作

* 定义求导类GradNet

* 求导类的__init__中定义需要求导的网络self.net和ops.GradOperation操作

* 求导类的construct函数中对self.net进行求导

$$f^{'}(x)=\omega$$

In [37]:
import mindspore.ops as ops
from numpy import gradient

class GradNet(nn.Cell):
    def __init__(self,net):
        super(GradNet,self).__init__()
        self.net=net
        self.grad_op=ops.GradOperation()
    
    def construct(self,x):
        gradient_func=self.grad_op(self.net)
        return gradient_func

x=ms.Tensor([100],dtype=ms.float32)
qd=GradNet(Net())(x)
print(qd)





DeadNode


此处输出DeadNode，是因为我们返回的是导函数而不是导函数值，return gradient_func应改为return gradient_func(x)

In [38]:
import mindspore.ops as ops
from numpy import gradient

class GradNet(nn.Cell):
    def __init__(self,net):
        super(GradNet,self).__init__()
        self.net=net
        self.grad_op=ops.GradOperation()
    
    def construct(self,x):
        gradient_func=self.grad_op(self.net)
        return gradient_func(x)

x=ms.Tensor([100],dtype=ms.float32)
qd=GradNet(Net())(x)
print(qd)

[6.]


### 对权重求一阶导

在很多场景之中，一个函数式往往包含多个参数，权重也有极大一部分是可学习的参数。故有时候我们也需要对权重进行求导

* 对权重参数求一阶导，需要将ops.GradOperation中的get_by_list设置为True(默认为False)

* 若某些权重不需要进行求导，则在定义求导网络时，相应的参数声明定义的时候将其requires_grad属性设置为False

In [43]:
import mindspore as ms

class GradNet(nn.Cell):
    def __init__(self,net):
        super(GradNet,self).__init__()
        self.net=net
        self.params=ms.ParameterTuple(net.trainable_params())
        self.grad_op=ops.GradOperation(get_by_list=True)

    def construct(self,x):
        gradient_func=self.grad_op(self.net,self.params)
        return gradient_func(x)

这里介绍一下ops.GradOperation的一些输入参数：

* **get_all**：计算梯度，如果等于False，获得第一个输入的梯度，如果等于True

* **get_by_list**：是否对权重的参数进行求导，默认值为False

* **sens_param**：是否对网络的输出值做缩放以改变最终梯度，默认值为False

接下来，对函数进行求导

In [44]:
x=ms.Tensor([100],dtype=ms.float32)
ds=GradNet(Net())(x)

print(ds)

print(f"wgrad:{ds[0]},\nbgrad{ds[1]}")

(Tensor(shape=[1], dtype=Float32, value= [1.00000000e+002]), Tensor(shape=[1], dtype=Float32, value= [1.00000000e+000]))
wgrad:[100.],
bgrad[1.]


那么如果当某一个某一个权重不需要求导，我们只需在创建该权重参数的地方将其**requires_grad**属性改为False即可